In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [2]:
import json

In [3]:
with open('semisupervised-pasentran-turki.json') as fopen:
    data = json.load(fopen)

In [4]:
len(data)

11789

In [5]:
import malaya_speech
import malaya

`pyaudio` is not available, `malaya_speech.streaming.pyaudio` is not able to use.


In [6]:
true_case = malaya.true_case.huggingface(model = 'mesolitica/finetune-true-case-t5-small-standard-bahasa-cased')

In [7]:
_ = true_case.cuda()

In [8]:
model = malaya_speech.force_alignment.transducer.transformer(device = 'gpu:0')

2023-02-28 21:36:45.760984: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-28 21:36:45.761615: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-28 21:36:45.763956: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-28 21:36:45.764683: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

In [9]:
def put(alignment, length, min_threshold = 0.5, min_threshold_fullstop = 1.0):
    r = []
    for no, row in enumerate(alignment):
        if no > 0:
            if alignment[no]['start'] - alignment[no-1]['end'] >= min_threshold:
                r.append(',')

        r.append(row['text'])
    if length - alignment[-1]['end'] >= min_threshold_fullstop:
        r.append('.')
    else:
        r.append(',')
    return r

In [11]:
from tqdm import tqdm

In [12]:
new_results = {}
for k, v in tqdm(data.items()):
    y, sr = malaya_speech.load(k)
    result = model.predict(y, v)
    if ' '.join([r['text'] for r in result['words_alignment']]) != v:
        continue
    o = put(result['words_alignment'], len(y) / sr)
    o = ' '.join(o)
    o_true_case = true_case.generate([o], max_length = 256)[0]
    
    new_results[k] = {
        'text': v,
        'force_alignment': o,
        'true_case': o_true_case,
    }

  0%|                                                                                                                           | 0/11789 [00:00<?, ?it/s]2023-02-28 21:36:59.894907: I tensorflow/stream_executor/cuda/cuda_blas.cc:1760] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-02-28 21:36:59.995187: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8302
2023-02-28 21:37:00.838913: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 11789/11789 [54:58<00:00,  3.57it/s]


In [13]:
with open('true-case-pasentran-turki.json', 'w') as fopen:
    json.dump(new_results, fopen)